# CREACIÓN DATAFRAME DATOS PRECIPITACIÓN (Est. puente carretera)

In [1]:
# Importación de Librerías necesarias
import numpy as np
import pandas as pd
import os

Los datos que se obtuvieron para esta estación provienen de la plataforma del IDEAM. Sin embargo, estos únicamente estaban disponibles en formato de precipitación acumulada cada 10 minutos y se tenían que descargar mes a mes. Se realizó la descarga de los datos desde enero de 2020 hasta marzo de 2022, periodo hasta el cual se contaba con datos disponibles.

Por lo tanto, el código que se presenta a continuación tiene como objetivo navegar a través de cada carpeta anual y leer, en orden, los datos correspondientes a cada mes. Además, es importante señalar que el código transformará la acumulación de precipitaciones de intervalos de 10 minutos a intervalos de una hora. Este ajuste es crucial para poder llevar a cabo la integración adecuada con la base de datos hidrológica.

In [2]:
# Ruta de la carpeta donde están los archivos
root_path = '../data/Precipitacion/'

# Crea una lista vacía para almacenar cada dataframe
dataframes = []

# Definimos el orden de los meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Define los encabezados que quieres leer desde el archivo csv
HEADERS = ['Fecha', 'Valor']

# Obtenemos una lista de todas las carpetas de años y las ordenamos
years = sorted([int(year) for year in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, year))])

# Recorremos los años en orden
for year in years:
    # Creamos la ruta a la carpeta del año
    folder_path = os.path.join(root_path, str(year))
    # Recorremos los meses en orden
    for mes in meses:
        # Recorremos todos los archivos en la carpeta
        for filename in os.listdir(folder_path):
            if filename.startswith(mes) and filename.endswith('.csv'):
                df = pd.read_csv(os.path.join(folder_path, filename), usecols=HEADERS)
                # Nos aseguramos de que la columna 'Fecha' esté en formato datetime
                df['Fecha'] = pd.to_datetime(df['Fecha']) 
                # Se reescala la frecuencia de los datos de 10 minutos a 1 hora, realizando la suma de los valores
                df = df.resample('H', on='Fecha').sum().reset_index()
                # Se añade el dataframe a la lista
                dataframes.append(df)

# Concatena todos los dataframes en uno solo
final_df = pd.concat(dataframes)
final_df = final_df.drop_duplicates(subset='Fecha')

El código que se proporcionará a continuación tiene como objetivo descomponer la columna 'Fecha' en dos columnas separadas: 'FECHA' y 'HORA'. Este paso es esencial ya que estas nuevas columnas servirán como puntos de referencia para realizar la fusión con el dataframe hidrológico.

Además, este código generará un archivo .csv. La creación de este archivo facilitará su lectura y manipulación en otros notebooks, permitiendo una mayor flexibilidad en su uso y análisis posterior.

In [3]:
def fecha_hora(df, nombre):
    # Convertir la columna de fecha a tipo datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])

    # Se crean nuevas series para Fecha y Hora
    fecha_nueva = df['Fecha'].copy()
    hora = df['Fecha'].dt.hour.copy()

    # Se ajustan las horas y fechas
    mask = hora == 0
    fecha_nueva[mask] = fecha_nueva[mask] - pd.Timedelta(days=1)
    hora[mask] = 24

    # Se actualiza las columnas de Fecha y Hora en el dataframe
    df['Fecha'] = fecha_nueva.dt.date
    df['Hora'] = hora

    # Se formatea la columna de fecha según lo requerido
    df['Fecha'] = df['Fecha'].astype(str)

    # Se renombran las columnas 'Fecha' a 'FECHA' y 'Hora' a 'HORA'
    df = df.rename(columns={'Fecha': 'FECHA', 'Hora': 'HORA'})
    df = df[['Valor', 'FECHA', 'HORA']]
    df = df.rename(columns={'Valor': nombre})

    # Se guarda el dataframe en un archivo csv
    df.to_csv(f'../data/{nombre}.csv', sep=',', index=False)

    return df

In [4]:
precipitacion = fecha_hora(final_df,'PRECIPITACION2')

In [5]:
precipitacion

,PRECIPITACION2,FECHA,HORA
0,1.2,2019-12-31,24
1,0.6,2020-01-01,1
2,0.0,2020-01-01,2
3,0.2,2020-01-01,3
4,0.0,2020-01-01,4
...,...,...,...
8035,0.0,2022-12-03,19
8036,0.0,2022-12-03,20
8037,0.0,2022-12-03,21
8038,0.0,2022-12-03,22
